In [2]:
import pandas as pd
import numpy as np
import scipy.stats as sts
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from catboost import CatBoostRegressor

import warnings
warnings.filterwarnings('ignore')

In [5]:
df = pd.read_csv('auto_ru.csv')
df = df.drop(columns=['Название']) #выкинули столбец "Название", так как столбцы "Марка" и "Модель" повторяют его содержимое

df.head()

,Марка,Модель,"Пробег, км","Мощность двигателя, л","Мощность двигателя, л.с",Год выпуска,"Цена, руб",Цвет кузова,Коробка передач,Привод,Тип топлива,Город продажи
0,BMW,3 серии 320d xDrive VII (G2x),40000,2,190,2020,3460000,оранжевый,автомат,полный,Дизель,Санкт-Петербург
1,BMW,3 серии 330d VII (G2x),80000,3,286,2019,3350000,синий,автомат,задний,Дизель,Санкт-Петербург
2,BMW,5 серии 530d xDrive VII (G30/G31),114000,3,249,2017,3960000,синий,автомат,полный,Дизель,Челябинск
3,BMW,3 серии 320d xDrive VII (G2x),42000,2,190,2021,4500000,белый,автомат,полный,Дизель,Москва
4,Audi,A6 45 TDI V (C8),38000,3,249,2018,5500000,белый,автомат,полный,Дизель,Санкт-Петербург


# 4. Создание новых переменных.

Создадим новый признак, коэффициент использования: пробег, делённый на возраст автомобиля.

In [7]:
df['Коэффициент использования'] = df['Пробег, км']  / ((2024 - df['Год выпуска']) * 1000)

Объединим модели в модельные ряды.

In [4]:
df['Модель'] = df['Модель'].str.split(' ').apply(lambda x: ' '.join(x[:1]))

Объединим редко встречающиеся цвета.

In [5]:
freq_uniq_color = df['Цвет кузова'].value_counts()
rare_colors = list(freq_uniq_color[freq_uniq_color < 250].index)
df['Цвет кузова'] = df['Цвет кузова'].apply(lambda color: 'редкий' if color in rare_colors else color)

Поработаем с признаком `Город продажи`:

- Объединим Подмосковье и Москву.
- Объединим иностранные города.
- Объединим относительно редкие города России.

In [6]:
moscow = ['Москва и Московская область', 'Химки', 'Балашиха', 'Красногорск', 'Одинцово', 'Люберцы', 'Зеленоград', 'Видное', 
          'Истра', 'Домодедово','Долгопрудный', 'Раменское', 'Лобня', 'Реутов', 'Солнечногорск', 'Орехово-Зуево', 'Коммунарка']
df['Город продажи'] = df['Город продажи'].replace(moscow, 'Москва')

foreign_cities = ['Минск', 'Гродно', 'Витебск', 'Гомель', 'Алматы', 'Астана', 'Костанай', 'Пинск', 'Беларусь', 
                  'Бишкек', 'Актобе', 'Шымкент']
df['Город продажи'] = df['Город продажи'].replace(foreign_cities, 'Заграница')

freq_uniq_cities = df['Город продажи'].value_counts()
rare_cities = list(freq_uniq_cities[freq_uniq_cities < 100].index)
df['Город продажи'] = df['Город продажи'].apply(lambda city: 'Глубинка' if city in rare_cities else city)

Закодируем категориальные признаки.

In [7]:
df = pd.get_dummies(df, columns=['Марка', 'Коробка передач', 'Тип топлива', 'Привод', 'Цвет кузова', 'Модель', 'Город продажи'])

In [8]:
df.head()

,"Пробег, км","Мощность двигателя, л","Мощность двигателя, л.с",Год выпуска,"Цена, руб",Коэффициент использования,Марка_Audi,Марка_BMW,Марка_Hyundai,Марка_Kia,...,Модель_Solaris,Город продажи_Владивосток,Город продажи_Глубинка,Город продажи_Екатеринбург,Город продажи_Заграница,Город продажи_Казань,Город продажи_Краснодар,Город продажи_Москва,Город продажи_Ростов-на-Дону,Город продажи_Санкт-Петербург
0,40000,2.0,190,2020,3460000,10.000000,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
1,80000,3.0,286,2019,3350000,16.000000,0,1,0,0,...,0,0,0,0,0,0,0,0,0,1
2,114000,3.0,249,2017,3960000,16.285714,0,1,0,0,...,0,0,1,0,0,0,0,0,0,0
3,42000,2.0,190,2021,4500000,14.000000,0,1,0,0,...,0,0,0,0,0,0,0,1,0,0
4,38000,3.0,249,2018,5500000,6.333333,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [9]:
df.columns

Index(['Пробег, км', 'Мощность двигателя, л', 'Мощность двигателя, л.с',
       'Год выпуска', 'Цена, руб', 'Коэффициент использования', 'Марка_Audi',
       'Марка_BMW', 'Марка_Hyundai', 'Марка_Kia', 'Марка_Mazda',
       'Марка_Mercedes-Benz', 'Коробка передач_автомат',
       'Коробка передач_вариатор', 'Коробка передач_механика',
       'Коробка передач_робот', 'Тип топлива_Бензин', 'Тип топлива_Газ',
       'Тип топлива_Гибрид', 'Тип топлива_Дизель', 'Привод_задний',
       'Привод_передний', 'Привод_полный', 'Цвет кузова_белый',
       'Цвет кузова_редкий', 'Цвет кузова_серый', 'Цвет кузова_синий',
       'Цвет кузова_чёрный', 'Модель_3', 'Модель_5', 'Модель_6', 'Модель_A4',
       'Модель_A6', 'Модель_CLA', 'Модель_Cerato', 'Модель_E-Класс',
       'Модель_Elantra', 'Модель_K5', 'Модель_S-Класс', 'Модель_Solaris',
       'Город продажи_Владивосток', 'Город продажи_Глубинка',
       'Город продажи_Екатеринбург', 'Город продажи_Заграница',
       'Город продажи_Казань', 'Город про

# 5. Гипотезы.

In [10]:
a = 0.05

def norm(x1, x2, col): #тест Шапиро — Уилка на нормальность распрделения
    
    s1 = sts.shapiro(x1[col])[1]
    s2 = sts.shapiro(x2[col])[1]
    
    if s1 < a: print('Распределение первой выборки не нормальное')
    else: print('Распределение первой выборки нормальное')
        
    if s2 < a: print('Распределение второй выборки не нормальное')
    else: print('Распределение второй выборки нормальное')
        
def u_test(x1, x2, col, how): # u-test
    p = sts.mannwhitneyu(x1[col], x2[col], alternative=how)[1]
    
    if p < a: print('Гипотеза подтверждается')
    else: print('Гипотеза отвергается')
        
def t_test(x1, x2, col, how): #t-test
    p = sts.ttest_ind(x1[col], x2[col], alternative=how)[1]
    
    if p < a: print('Гипотеза подтверждается')
    else: print('Гипотеза отвергается')

**№1: машины на дизельном топливе используют активнее, чем на бензине.**

In [11]:
df1 = df[df['Тип топлива_Дизель'] == 1]
df2 = df[df['Тип топлива_Бензин'] == 1]

In [12]:
norm(df1, df2, 'Коэффициент использования')

Распределение первой выборки не нормальное
Распределение второй выборки не нормальное


Значит нужно использовать u-test.

In [13]:
u_test(df1, df2, 'Коэффициент использования', 'greater')

Гипотеза подтверждается


**№2: машины с вариатором и роботом используюся менее активно, чем машины с механикой и автоматом.**

In [14]:
df1 = df[(df['Коробка передач_вариатор'] == 1) | (df['Коробка передач_робот'] == 1)]
df2 = df[(df['Коробка передач_автомат'] == 1) | (df['Коробка передач_механика'] == 1)]

In [15]:
norm(df1, df2, 'Коэффициент использования')

Распределение первой выборки не нормальное
Распределение второй выборки не нормальное


In [16]:
u_test(df1, df2, 'Коэффициент использования', 'less')

Гипотеза подтверждается


**№3: мерседесы дороже других автомобилей.**

In [17]:
df1 = df[df['Марка_Mercedes-Benz'] == 1]
df2 = df[df['Марка_Mercedes-Benz'] == 0]

In [18]:
norm(df1, df2, 'Цена, руб')

Распределение первой выборки не нормальное
Распределение второй выборки не нормальное


In [19]:
u_test(df1, df2, 'Цена, руб', 'greater')

Гипотеза подтверждается


**№4: машины заграницей дешевле.**

In [20]:
df1 = df[df['Город продажи_Заграница'] == 1]
df2 = df[df['Город продажи_Заграница'] == 0]

In [21]:
norm(df1, df2, 'Цена, руб')

Распределение первой выборки не нормальное
Распределение второй выборки не нормальное


In [22]:
u_test(df1, df2, 'Цена, руб', 'less')

Гипотеза отвергается


**№5: во Владивостоке машины азиатских брендов дешевле, чем в других городах.**

In [23]:
df1 = df[(df['Город продажи_Владивосток'] == 1) & 
         ((df['Марка_Hyundai'] == 1) | (df['Марка_Kia'] == 1) | (df['Марка_Mazda'] == 1))]
df2 = df[(df['Город продажи_Владивосток'] == 0) & 
         ((df['Марка_Hyundai'] == 1) | (df['Марка_Kia'] == 1) | (df['Марка_Mazda'] == 1))]

In [24]:
norm(df1, df2, 'Цена, руб')

Распределение первой выборки нормальное
Распределение второй выборки не нормальное


Испольуем t-тест.

In [25]:
t_test(df1, df2, 'Цена, руб', 'less')

Гипотеза отвергается


**№6: дизельные двигатели имеют больше лошадиных сил, чем иные двигатели.**

In [28]:
df1 = df[df['Тип топлива_Дизель'] == 1]
df2 = df[df['Тип топлива_Дизель'] == 0]

In [29]:
norm(df1, df2, 'Мощность двигателя, л.с')

Распределение первой выборки не нормальное
Распределение второй выборки не нормальное


In [30]:
u_test(df1, df2, 'Мощность двигателя, л.с', 'greater')

Гипотеза подтверждается


**№7 машины редких цветов стоят дороже обычных.**

In [31]:
df1 = df[df['Цвет кузова_редкий'] == 1]
df2 = df[df['Цвет кузова_редкий'] == 0]

In [32]:
norm(df1, df2, 'Цена, руб')

Распределение первой выборки не нормальное
Распределение второй выборки не нормальное


In [33]:
u_test(df1, df2, 'Цена, руб', 'greater')

Гипотеза отвергается


**№8 Машины с механикой отличаются по цене от машин с автоматом.**

In [34]:
df1 = df[df['Коробка передач_механика'] == 1]
df2 = df[df['Коробка передач_автомат'] == 1]

In [35]:
norm(df1, df2, 'Цена, руб')

Распределение первой выборки не нормальное
Распределение второй выборки не нормальное


In [36]:
u_test(df1, df2, 'Цена, руб', 'two-sided')

Гипотеза подтверждается


# 6. Машинное обучение.

Будем предсказывать цену автомобиля. Так как у нас много категориальных признаков, то эффективнее использовать нелинейные модели. Для оценки точности модели будем использовать MAE. 

Подготовим данные:

In [37]:
df_train, df_test = train_test_split(df, test_size=0.2, random_state=14)

X_train = df_train.drop('Цена, руб', axis=1)
y_train = df_train['Цена, руб']

X_test = df_test.drop('Цена, руб', axis=1)
y_test = df_test['Цена, руб']

columns = X_train.columns
scaler = MinMaxScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=columns, index=X_train.index)
X_test = pd.DataFrame(scaler.fit_transform(X_test), columns=columns, index=X_test.index)

In [38]:
def quality(result): # рассчет метрики MAE
    mae = mean_absolute_error(y_test, result)
    print('Средняя абсолютная ошибка: ' + str(round(mae)) + ' рублей или ' + str(round(mae/y_test.mean() * 100)) +'%')

Используем модель случайного леса.

In [39]:
random_forest = RandomForestRegressor()
random_forest.fit(X_train, y_train)

quality(random_forest.predict(X_test))

Средняя абсолютная ошибка: 470434 рублей или 14%


Попробуем подобрать оптимальные параметры для случайного леса.

In [214]:
params = {
    'n_estimators': [100, 300, 500],
    'max_depth': [3, 5, 7, 9],
    'min_samples_split': [2, 3, 5, 7],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

grid_search = GridSearchCV(random_forest, params, scoring='neg_mean_absolute_error', cv = 5, n_jobs = -1)
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)

{'max_depth': 9, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 7, 'n_estimators': 300}


In [40]:
best_random_forest = RandomForestRegressor(max_depth = 9, max_features = 'auto', min_samples_leaf = 1, 
                                           min_samples_split = 7, n_estimators = 300)
best_random_forest.fit(X_train, y_train)
quality(best_random_forest.predict(X_test))

Средняя абсолютная ошибка: 472558 рублей или 14%


Особо ничего не изменилось. Попробуем использовать кэтбуст.

In [219]:
params = {
    'iterations': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'depth': [3, 5, 7, 9],
}

cat_boost = CatBoostRegressor()

grid_search = GridSearchCV(estimator=cat_boost, param_grid=params, cv=5)
grid_search.fit(X_train, y_train)

print(grid_search.best_params_)

0:	learn: 2664483.5828572	total: 165ms	remaining: 16.3s
1:	learn: 2642423.4234977	total: 167ms	remaining: 8.18s
2:	learn: 2620942.4683393	total: 168ms	remaining: 5.44s
3:	learn: 2599746.3546519	total: 170ms	remaining: 4.07s
4:	learn: 2578974.9335589	total: 171ms	remaining: 3.24s
5:	learn: 2558102.7162957	total: 172ms	remaining: 2.69s
6:	learn: 2537733.9549258	total: 173ms	remaining: 2.3s
7:	learn: 2517360.0491107	total: 174ms	remaining: 2.01s
8:	learn: 2497091.6543998	total: 176ms	remaining: 1.78s
9:	learn: 2476961.6020988	total: 177ms	remaining: 1.59s
10:	learn: 2457494.1380919	total: 178ms	remaining: 1.44s
11:	learn: 2437656.2054074	total: 179ms	remaining: 1.31s
12:	learn: 2418759.4398719	total: 180ms	remaining: 1.2s
13:	learn: 2399717.8312561	total: 181ms	remaining: 1.11s
14:	learn: 2380866.5898045	total: 182ms	remaining: 1.03s
15:	learn: 2362851.0328242	total: 183ms	remaining: 960ms
16:	learn: 2345223.9121588	total: 184ms	remaining: 897ms
17:	learn: 2326724.1245110	total: 185ms	rem

23:	learn: 2292126.0574947	total: 25.7ms	remaining: 81.3ms
24:	learn: 2274753.3291894	total: 27.1ms	remaining: 81.2ms
25:	learn: 2257715.5567074	total: 28.2ms	remaining: 80.4ms
26:	learn: 2240967.3250958	total: 29.4ms	remaining: 79.5ms
27:	learn: 2224746.6555172	total: 30.5ms	remaining: 78.4ms
28:	learn: 2208364.3496527	total: 31.5ms	remaining: 77.2ms
29:	learn: 2192691.7554037	total: 32.7ms	remaining: 76.2ms
30:	learn: 2177102.5660990	total: 33.8ms	remaining: 75.3ms
31:	learn: 2161083.2969329	total: 34.9ms	remaining: 74.2ms
32:	learn: 2145256.8522684	total: 36.2ms	remaining: 73.5ms
33:	learn: 2129803.1440144	total: 37.4ms	remaining: 72.7ms
34:	learn: 2115115.2369779	total: 38.7ms	remaining: 71.9ms
35:	learn: 2100059.3635147	total: 40ms	remaining: 71.2ms
36:	learn: 2085383.2854800	total: 41.3ms	remaining: 70.3ms
37:	learn: 2070912.7831813	total: 42.4ms	remaining: 69.2ms
38:	learn: 2056224.8739320	total: 43.6ms	remaining: 68.2ms
39:	learn: 2041566.0063072	total: 44.9ms	remaining: 67.3ms

63:	learn: 1696930.7677640	total: 71.6ms	remaining: 40.3ms
64:	learn: 1685829.7168419	total: 72.8ms	remaining: 39.2ms
65:	learn: 1675099.4548660	total: 74ms	remaining: 38.1ms
66:	learn: 1663999.8968454	total: 75.1ms	remaining: 37ms
67:	learn: 1653014.3050926	total: 76.2ms	remaining: 35.9ms
68:	learn: 1642651.4086436	total: 77.4ms	remaining: 34.8ms
69:	learn: 1632429.1054998	total: 78.5ms	remaining: 33.6ms
70:	learn: 1622136.6667230	total: 79.6ms	remaining: 32.5ms
71:	learn: 1611949.8299439	total: 80.7ms	remaining: 31.4ms
72:	learn: 1602057.8992829	total: 81.9ms	remaining: 30.3ms
73:	learn: 1592160.9912552	total: 83.2ms	remaining: 29.2ms
74:	learn: 1582284.2086835	total: 84.3ms	remaining: 28.1ms
75:	learn: 1572603.8550328	total: 85.5ms	remaining: 27ms
76:	learn: 1562945.3719011	total: 86.6ms	remaining: 25.9ms
77:	learn: 1554103.6723339	total: 87.7ms	remaining: 24.7ms
78:	learn: 1544758.3042314	total: 88.7ms	remaining: 23.6ms
79:	learn: 1535508.6196771	total: 89.7ms	remaining: 22.4ms
80:

80:	learn: 831110.6374799	total: 99.1ms	remaining: 23.2ms
81:	learn: 829213.2017626	total: 100ms	remaining: 22ms
82:	learn: 826940.5730913	total: 101ms	remaining: 20.7ms
83:	learn: 825139.3310257	total: 102ms	remaining: 19.5ms
84:	learn: 823731.6984317	total: 104ms	remaining: 18.3ms
85:	learn: 822382.0598327	total: 105ms	remaining: 17.1ms
86:	learn: 821026.0767091	total: 106ms	remaining: 15.8ms
87:	learn: 819306.1035388	total: 107ms	remaining: 14.6ms
88:	learn: 817772.2885316	total: 108ms	remaining: 13.4ms
89:	learn: 816693.7409588	total: 109ms	remaining: 12.2ms
90:	learn: 815417.6815569	total: 111ms	remaining: 10.9ms
91:	learn: 814300.8153366	total: 112ms	remaining: 9.72ms
92:	learn: 813367.0659320	total: 113ms	remaining: 8.52ms
93:	learn: 812016.5847229	total: 114ms	remaining: 7.31ms
94:	learn: 810877.9121973	total: 116ms	remaining: 6.09ms
95:	learn: 809582.4343307	total: 117ms	remaining: 4.87ms
96:	learn: 808805.9257670	total: 118ms	remaining: 3.66ms
97:	learn: 807502.8111700	total:

0:	learn: 2620107.6962168	total: 11.4ms	remaining: 1.13s
1:	learn: 2512037.5783982	total: 12.8ms	remaining: 625ms
2:	learn: 2411702.2582090	total: 13.9ms	remaining: 450ms
3:	learn: 2317414.5900804	total: 15ms	remaining: 360ms
4:	learn: 2228234.9103993	total: 16ms	remaining: 304ms
5:	learn: 2142488.2565199	total: 16.9ms	remaining: 266ms
6:	learn: 2062262.1624201	total: 18ms	remaining: 239ms
7:	learn: 1984872.4732092	total: 19.3ms	remaining: 222ms
8:	learn: 1911735.5119853	total: 20.4ms	remaining: 207ms
9:	learn: 1843978.3753766	total: 21.6ms	remaining: 194ms
10:	learn: 1779768.5458159	total: 22.9ms	remaining: 185ms
11:	learn: 1719260.4956194	total: 24.1ms	remaining: 177ms
12:	learn: 1662422.8375234	total: 25.4ms	remaining: 170ms
13:	learn: 1611088.4562919	total: 26.5ms	remaining: 163ms
14:	learn: 1559467.1825429	total: 27.5ms	remaining: 156ms
15:	learn: 1512297.4225999	total: 28.5ms	remaining: 150ms
16:	learn: 1471243.9015397	total: 29.5ms	remaining: 144ms
17:	learn: 1430701.9258955	tot

...

65:	learn: 603518.2124357	total: 395ms	remaining: 1.4s
66:	learn: 601040.1575493	total: 401ms	remaining: 1.4s
67:	learn: 599264.4043068	total: 407ms	remaining: 1.39s
68:	learn: 594991.8237206	total: 413ms	remaining: 1.38s
69:	learn: 593058.1067153	total: 419ms	remaining: 1.38s
70:	learn: 590657.0936014	total: 424ms	remaining: 1.37s
71:	learn: 587436.6983401	total: 430ms	remaining: 1.36s
72:	learn: 585889.9019014	total: 435ms	remaining: 1.35s
73:	learn: 582830.5111961	total: 442ms	remaining: 1.35s
74:	learn: 581261.6193834	total: 447ms	remaining: 1.34s
75:	learn: 579469.4088127	total: 453ms	remaining: 1.33s
76:	learn: 576871.2647428	total: 459ms	remaining: 1.33s
77:	learn: 575167.3477535	total: 464ms	remaining: 1.32s
78:	learn: 572503.4868710	total: 469ms	remaining: 1.31s
79:	learn: 570356.1991803	total: 475ms	remaining: 1.31s
80:	learn: 569513.2217686	total: 477ms	remaining: 1.29s
81:	learn: 567420.3947328	total: 482ms	remaining: 1.28s
82:	learn: 565407.8156146	total: 488ms	remaining: 

226:	learn: 397764.4826230	total: 1.32s	remaining: 426ms
227:	learn: 397104.1942903	total: 1.33s	remaining: 421ms
228:	learn: 396329.1157029	total: 1.34s	remaining: 415ms
229:	learn: 395449.8429300	total: 1.34s	remaining: 409ms
230:	learn: 394511.7926325	total: 1.35s	remaining: 403ms
231:	learn: 393387.1906419	total: 1.36s	remaining: 397ms
232:	learn: 392879.8779039	total: 1.36s	remaining: 392ms
233:	learn: 392069.6465729	total: 1.37s	remaining: 386ms
234:	learn: 390912.5701124	total: 1.37s	remaining: 380ms
235:	learn: 390214.1758569	total: 1.38s	remaining: 374ms
236:	learn: 389717.6908616	total: 1.39s	remaining: 368ms
237:	learn: 389123.0957277	total: 1.39s	remaining: 363ms
238:	learn: 388255.5097643	total: 1.4s	remaining: 357ms
239:	learn: 387084.1100804	total: 1.4s	remaining: 351ms
240:	learn: 386540.1022624	total: 1.41s	remaining: 345ms
241:	learn: 386022.1335339	total: 1.41s	remaining: 339ms
242:	learn: 385025.7195752	total: 1.42s	remaining: 333ms
243:	learn: 384392.7447103	total:

98:	learn: 508081.3495469	total: 593ms	remaining: 1.2s
99:	learn: 506349.4762568	total: 599ms	remaining: 1.2s
100:	learn: 505065.1668873	total: 605ms	remaining: 1.19s
101:	learn: 503365.4848345	total: 612ms	remaining: 1.19s
102:	learn: 502057.8330615	total: 617ms	remaining: 1.18s
103:	learn: 500107.1650259	total: 623ms	remaining: 1.17s
104:	learn: 498400.8877468	total: 629ms	remaining: 1.17s
105:	learn: 497209.0961574	total: 635ms	remaining: 1.16s
106:	learn: 495886.9125470	total: 640ms	remaining: 1.16s
107:	learn: 494254.8590314	total: 646ms	remaining: 1.15s
108:	learn: 493472.9326686	total: 652ms	remaining: 1.14s
109:	learn: 492693.3367815	total: 658ms	remaining: 1.14s
110:	learn: 491168.7876833	total: 665ms	remaining: 1.13s
111:	learn: 489742.9687310	total: 671ms	remaining: 1.13s
112:	learn: 488163.7402083	total: 677ms	remaining: 1.12s
113:	learn: 486332.9497742	total: 683ms	remaining: 1.11s
114:	learn: 484359.3362001	total: 690ms	remaining: 1.11s
115:	learn: 483433.0264625	total: 6

251:	learn: 383249.3180229	total: 1.49s	remaining: 284ms
252:	learn: 382816.7862022	total: 1.5s	remaining: 278ms
253:	learn: 382535.8562229	total: 1.5s	remaining: 272ms
254:	learn: 381861.1730291	total: 1.51s	remaining: 266ms
255:	learn: 381444.5387743	total: 1.51s	remaining: 260ms
256:	learn: 380910.7790558	total: 1.52s	remaining: 255ms
257:	learn: 380375.3203802	total: 1.53s	remaining: 249ms
258:	learn: 380059.2087711	total: 1.53s	remaining: 243ms
259:	learn: 379245.1506708	total: 1.54s	remaining: 237ms
260:	learn: 378489.7220926	total: 1.54s	remaining: 231ms
261:	learn: 378156.1141103	total: 1.55s	remaining: 225ms
262:	learn: 377368.5214295	total: 1.56s	remaining: 219ms
263:	learn: 376753.5981901	total: 1.56s	remaining: 213ms
264:	learn: 376241.4765979	total: 1.57s	remaining: 207ms
265:	learn: 375593.6084364	total: 1.57s	remaining: 201ms
266:	learn: 375274.4205235	total: 1.58s	remaining: 195ms
267:	learn: 374779.9884032	total: 1.58s	remaining: 189ms
268:	learn: 374316.5902060	total:

98:	learn: 649719.8550727	total: 582ms	remaining: 594ms
99:	learn: 647541.5598870	total: 588ms	remaining: 588ms
100:	learn: 646792.8002196	total: 595ms	remaining: 583ms
101:	learn: 645307.3169580	total: 601ms	remaining: 578ms
102:	learn: 643357.2075775	total: 607ms	remaining: 571ms
103:	learn: 641757.4607731	total: 613ms	remaining: 565ms
104:	learn: 640463.5872406	total: 619ms	remaining: 560ms
105:	learn: 639450.0403138	total: 625ms	remaining: 554ms
106:	learn: 638242.2990412	total: 630ms	remaining: 548ms
107:	learn: 636752.3404784	total: 636ms	remaining: 542ms
108:	learn: 634267.9643824	total: 643ms	remaining: 536ms
109:	learn: 632780.1504924	total: 648ms	remaining: 530ms
110:	learn: 631643.5248547	total: 654ms	remaining: 524ms
111:	learn: 630469.5709409	total: 660ms	remaining: 518ms
112:	learn: 629319.4149678	total: 666ms	remaining: 513ms
113:	learn: 627720.3436368	total: 671ms	remaining: 506ms
114:	learn: 626160.2383031	total: 677ms	remaining: 500ms
115:	learn: 625193.1685606	total:

In [41]:
best_cat_boost = CatBoostRegressor(depth = 9, iterations = 200, learning_rate = 0.05)
best_cat_boost.fit(X_train, y_train)

quality(best_cat_boost.predict(X_test))

0:	learn: 2622916.3903153	total: 127ms	remaining: 25.2s
1:	learn: 2521736.0957214	total: 138ms	remaining: 13.6s
2:	learn: 2423461.2706489	total: 147ms	remaining: 9.68s
3:	learn: 2324490.2337289	total: 155ms	remaining: 7.58s
4:	learn: 2231997.3258584	total: 166ms	remaining: 6.46s
5:	learn: 2143917.4791432	total: 176ms	remaining: 5.68s
6:	learn: 2064433.2369181	total: 187ms	remaining: 5.14s
7:	learn: 1985154.9899569	total: 198ms	remaining: 4.74s
8:	learn: 1910007.5069867	total: 203ms	remaining: 4.3s
9:	learn: 1842315.8486770	total: 213ms	remaining: 4.05s
10:	learn: 1778806.5888341	total: 224ms	remaining: 3.85s
11:	learn: 1714945.3241833	total: 234ms	remaining: 3.66s
12:	learn: 1657642.0961961	total: 245ms	remaining: 3.52s
13:	learn: 1598728.1672850	total: 255ms	remaining: 3.39s
14:	learn: 1544718.8228618	total: 266ms	remaining: 3.29s
15:	learn: 1495046.7406115	total: 278ms	remaining: 3.2s
16:	learn: 1453490.7657933	total: 289ms	remaining: 3.11s
17:	learn: 1407145.7449133	total: 300ms	rem

161:	learn: 578296.5264755	total: 1.81s	remaining: 425ms
162:	learn: 577019.8565272	total: 1.82s	remaining: 414ms
163:	learn: 575834.0291340	total: 1.83s	remaining: 403ms
164:	learn: 575178.7439186	total: 1.84s	remaining: 391ms
165:	learn: 574234.3903498	total: 1.85s	remaining: 380ms
166:	learn: 573755.6109330	total: 1.87s	remaining: 369ms
167:	learn: 573320.8130025	total: 1.87s	remaining: 356ms
168:	learn: 572819.6242006	total: 1.88s	remaining: 345ms
169:	learn: 571845.8520125	total: 1.89s	remaining: 334ms
170:	learn: 570569.2535440	total: 1.9s	remaining: 323ms
171:	learn: 569546.9514022	total: 1.91s	remaining: 312ms
172:	learn: 569131.4658619	total: 1.92s	remaining: 299ms
173:	learn: 567961.1559027	total: 1.93s	remaining: 288ms
174:	learn: 567463.6949210	total: 1.94s	remaining: 277ms
175:	learn: 566496.2303197	total: 1.95s	remaining: 266ms
176:	learn: 565567.8936658	total: 1.96s	remaining: 255ms
177:	learn: 565184.1159802	total: 1.97s	remaining: 243ms
178:	learn: 564275.7276139	total

Бустинг тоже не дал особого результата, оно и неудевительно -- случайный лес и кэтбуст похожи, но в любом случае ошибка в 14% - это хороший показатель в рамках данного проекта. Если смотреть на машины до 1 млн такая ошибка может показаться критичной. Однако, начиная от 5 млн такая ошибка в целом неплохой результат.

В исследовании мы используем ограниченное количество данных об автомобилях, возможно, удалось бы добиться лучшего качества, собрав и включив в модель дополнительные признаки, например: количество владельцев, информация о ДТП, состояние машины, размер налога, комплектация и т.д.